<a href="https://colab.research.google.com/github/anand-therattil/Fake_News_Detection/blob/Initial-Commit/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Modules

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sn
from pandas.core.common import random_state
import re 
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Printed stopwords to remove 
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [3]:
drive.mount(r'/content/drive')
train_data = pd.read_csv(r'drive/MyDrive/Colab Notebooks/Dataset/fake_news_train.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_data.head()
# label 1 Fake news
# label 0 Real News

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Data Pre-Processing

In [5]:
# Misssing Data
train_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
train_data= train_data.fillna("")
train_data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
dependent = train_data['label']
independent = train_data.drop(['label'],axis=1)
independent['content'] = independent['author']+independent['title']
independent.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard PortnoyIranian woman jailed for fiction...


In [8]:
port_stem = PorterStemmer()

In [9]:
def stemming(content):
  # String normalizer 
  stemmed_content = re.sub('[^a-z A-Z]',' ',content) # removes everything except the alphabets
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  stemmed_content  = ' '.join(stemmed_content)
  return stemmed_content

In [10]:
independent['content'] = independent['content'].apply(stemming)

In [11]:
independent.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,darrel lucushous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,daniel j flynnflynn hillari clinton big woman ...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",consortiumnew comwhi truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,howard portnoyiranian woman jail fiction unpub...


In [12]:
x= independent['content'].values
y = dependent.values
print(x)

['darrel lucushous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exercis balkan'
 'david swansonwhat keep f aliv']


In [13]:
# To vectorise the function as processing on
vectorizer = TfidfVectorizer()
x =vectorizer.fit_transform(x)
print(x)

  (0, 26340)	0.28088379401596425
  (0, 4211)	0.3625320323150658
  (0, 12782)	0.24619727512767195
  (0, 14555)	0.29177259684200296
  (0, 5006)	0.2472595823572816
  (0, 22724)	0.2552336018069161
  (0, 8022)	0.23133661742488731
  (0, 578)	0.2694167078545385
  (0, 6273)	0.2839932825877813
  (0, 15019)	0.43006226759639316
  (0, 5969)	0.35488202138141456
  (1, 3281)	0.18652439327549422
  (1, 3859)	0.45980466668763464
  (1, 27923)	0.3691184595384501
  (1, 2622)	0.3562953366945266
  (1, 4767)	0.23338756776626787
  (1, 11313)	0.24166773097712632
  (1, 8772)	0.525863562538645
  (1, 5916)	0.3181005810963805
  (2, 8567)	0.3411947414020896
  (2, 9454)	0.30743020569262086
  (2, 16361)	0.43295215406038445
  (2, 26235)	0.3665032495181434
  (2, 5121)	0.5511414848555652
  (2, 5240)	0.40440534260277944
  :	:
  (20797, 16217)	0.3273246827604847
  (20797, 1249)	0.3072223353708335
  (20797, 2257)	0.3357782642976524
  (20797, 22086)	0.24902354987792552
  (20797, 20778)	0.2729578683228216
  (20797, 25319)	0.3

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, stratify=y,random_state =2,)

In [15]:
model = LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [16]:
# Evalution on training
x_train_predict = model.predict(x_train)
print(accuracy_score(y_train,x_train_predict)*100)

97.19951923076923


In [17]:
# Evalution on testing
x_test_predict = model.predict(x_test)
print(accuracy_score(y_test,x_test_predict)*100)

95.48076923076924
